# Segmenting and Clustering Neighborhoods in Toronto part 3

## 1. Download and Explore Dataset

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = bs(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

d0 = df[0][0];d0 = d0[1 : ]
d1 = df[0][1];d1 = d1[1 : ]
d2 = df[0][2];d2 = d2[1 : ]
df1 = pd.DataFrame({'PostalCode':d0, 'Borough':d1, 'Neighborhood':d2})

df1.drop(df1[df1["Borough"] == "Not assigned"].index, axis = 0, inplace = True)
df = df1.groupby("PostalCode").agg(lambda x: ','.join(set(x)))
df.reset_index(level = 0, inplace = True)

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
df_a = df_data_1.drop('Postal Code', 1)
Toronto_neighborhoods = pd.concat([df, df_a], axis=1)
Toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Install libraries

In [5]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.61 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  35.79 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.92 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.54 MB/s
vincent-0.4.4- 100% |###################

In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_neighborhoods['Borough'].unique()),
        Toronto_neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


## Get toronto address

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_neighborhoods['Latitude'], Toronto_neighborhoods['Longitude'], 
                                           Toronto_neighborhoods['Borough'], Toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods and only boroughs that contain the word Toronto. So let's slice the original dataframe and create a new dataframe with only boroughs that contain the word Toronto.

In [9]:
Word_Toronto_data = Toronto_neighborhoods[Toronto_neighborhoods['Borough'].isin(['Central Toronto','Downtown Toronto','East Toronto','West Toronto'])].reset_index(drop=True)

Word_Toronto_data.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [10]:
# create map of Toronto using latitude and longitude values
Word_Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Word_Toronto_data['Latitude'], Word_Toronto_data['Longitude'], 
                                           Word_Toronto_data['Borough'], Word_Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Word_Toronto_map)  
    
Word_Toronto_map

# Define Foursquare Credentials and Version

In [11]:
# The code was removed by Watson Studio for sharing.

# Let's explore the second neighborhood in our dataframe.

Get the neighborhood's name.

In [12]:
Word_Toronto_data.loc[1, 'Neighborhood']

'The Danforth West,Riverdale'

Get the neighborhood's latitude and longitude values.

In [13]:
neighborhood_latitude = Word_Toronto_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Word_Toronto_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = Word_Toronto_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Danforth West,Riverdale are 43.6795571, -79.352188.


# Now, let's get the top 100 venues that are in Riverdale,The Danforth West within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [14]:
# type your answer here
search_query = neighborhood_name
radius = 500
LIMIT = 100
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, 
CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)

The Danforth West,Riverdale .... OK!


Send the GET request and examine the resutls

In [15]:
results = requests.get(url).json()

Now we are ready to clean the json and structure it into a pandas dataframe.

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Christina's On The Danforth,Greek Restaurant,43.678240,-79.349185
1,Pan on the Danforth,Greek Restaurant,43.678263,-79.348648
2,Danforth Medical Arts Clinic,Doctor's Office,43.676931,-79.355606
3,Danforth Medical Centre,Medical Center,43.676858,-79.355403


And how many venues were returned by Foursquare?

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Word_Toronto_data['Borough'].unique()),
        Word_Toronto_data.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


### Let's create a function to repeat the same process to all the neighborhoods with only boroughs that contain the word Toronto.

In [20]:
# The code was removed by Watson Studio for sharing.

Code to run the above function on each neighborhood and create a new dataframe called Word_Toronto_venues.

In [21]:
Word_Toronto_venues = getNearbyVenues(names=Word_Toronto_data['Neighborhood'],
                                   latitudes=Word_Toronto_data['Latitude'],
                                   longitudes=Word_Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Forest Hill SE,Deer Park,Summerhill West,Rathnelly,South Hill
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Regent Park,Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Union Station,Harbourfront East,Toronto Islands
Toronto Dominion Centre,Design Exchange
Victoria Hotel,Commerce Court
Roselawn
Forest Hill North,Forest Hill West
Yorkville,The Annex,North Midtown
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
Harbourfront West,CN Tower,King and Spadina,South Niagara,Railway Lands,Island airport,Bathurst Quay
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Parkdale Village,Exhibition Place,Brockton
High Park,The Junction South
Parkdale,Roncesvall

### Let's check the size of the resulting dataframe

In [22]:
print(Word_Toronto_venues.shape)
Word_Toronto_venues.head(2)

(1708, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub


Let's check how many venues were returned for each neighborhood

In [23]:
Word_Toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
Central Bay Street,81,81,81,81,81,81
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(Word_Toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


## 3. Analyze Each Neighborhood

In [25]:
# one hot encoding
Word_Toronto_onehot = pd.get_dummies(Word_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Word_Toronto_onehot['Neighborhood'] = Word_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Word_Toronto_onehot.columns[-1]] + list(Word_Toronto_onehot.columns[:-1])
Word_Toronto_onehot = Word_Toronto_onehot[fixed_columns]

Word_Toronto_onehot.head(2)

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [26]:
Word_Toronto_onehot.shape

(1708, 240)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
Word_Toronto_grouped = Word_Toronto_onehot.groupby('Neighborhood').mean().reset_index()

### Let's confirm the new size

In [28]:
Word_Toronto_grouped.shape

(38, 240)

### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in Word_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Word_Toronto_grouped[Word_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,Richmond,King----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.04
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3          Steakhouse  0.04
4      Farmers Market  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3                Café  0.04
4                 Bar  0.04


----Chinatown,Grange Park,Kensington Market----
                           venue  freq
0                           Café  0.07
1                            Bar  0.06
2          Vi

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Word_Toronto_grouped['Neighborhood']

for ind in np.arange(Word_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Word_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Steakhouse,Café,Thai Restaurant,American Restaurant,Bakery,Restaurant,Bar,Gym,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Café,Pub


# 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 5

Word_Toronto_grouped_clustering = Word_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Word_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Word_Toronto_merged = Word_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Word_Toronto_merged = Word_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Word_Toronto_merged.head(2) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Café


Finally, let's visualize the resulting clusters

In [116]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Word_Toronto_merged['Latitude'], Word_Toronto_merged['Longitude'], Word_Toronto_merged['Neighborhood'], Word_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

Now, let's can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

## Cluster 1

In [ ]:
Word_Toronto_merged.loc[Word_Toronto_merged['Cluster Labels'] == 0, Word_Toronto_merged.columns[[1] + list(range(5, Word_Toronto_merged.shape[1]))]]

## Cluster 2

In [77]:
# We don't want to print results
Word_Toronto_merged.loc[Word_Toronto_merged['Cluster Labels'] == 1, Word_Toronto_merged.columns[[1] + list(range(5, Word_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Colombian Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


## Cluster 3

In [78]:
Word_Toronto_merged.loc[Word_Toronto_merged['Cluster Labels'] == 2, Word_Toronto_merged.columns[[1] + list(range(5, Word_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Wings Joint,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## Cluster 4

In [79]:
Word_Toronto_merged.loc[Word_Toronto_merged['Cluster Labels'] == 3, Word_Toronto_merged.columns[[1] + list(range(5, Word_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Park,Playground,Trail,Wings Joint,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
23,Central Toronto,3,Bus Line,Park,Sushi Restaurant,Jewelry Store,Trail,Donut Shop,Discount Store,Dog Run,Doner Restaurant,Wings Joint


# Cluster 5

In [82]:
Word_Toronto_merged.loc[Word_Toronto_merged['Cluster Labels'] == 4, Word_Toronto_merged.columns[[1] + list(range(5, Word_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Swim School,Bus Line,Wings Joint,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
